In [1]:
import os
import sys
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Get current working directory instead of __file__
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from EfficientNetV2.Model import EfficientNetV2SWheatCountWithConfidence
from dataLoaderFunc import loadSplitData, createLoader


c:\Users\pacha\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ✅ Custom Gaussian NLL Loss
def gaussian_nll_loss(pred_mean, pred_logvar, target):
    precision = torch.exp(-pred_logvar)
    return torch.mean(precision * (target - pred_mean)**2 + pred_logvar)

# ✅ Training Function
def train_model(model, train_loader, val_loader, optimizer, scheduler, device, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for batch_idx, (rgb_batch, dsm_batch, label_batch) in enumerate(train_loader):
            rgb_batch, dsm_batch, label_batch = rgb_batch.to(device), dsm_batch.to(device), label_batch.to(device)

            optimizer.zero_grad()
            output = model(rgb_batch, dsm_batch)  # output: [B, 2]
            pred_mean = output[:, 0]
            pred_logvar = output[:, 1]
            loss = gaussian_nll_loss(pred_mean, pred_logvar, label_batch.squeeze())
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            if batch_idx % 20 == 0:
                print(f"Epoch {epoch+1}/{num_epochs} | Batch {batch_idx}/{len(train_loader)} | Loss: {loss.item():.4f}")

        train_loss /= len(train_loader)

        # ✅ Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for rgb_batch, dsm_batch, label_batch in val_loader:
                rgb_batch, dsm_batch, label_batch = rgb_batch.to(device), dsm_batch.to(device), label_batch.to(device)
                output = model(rgb_batch, dsm_batch)
                pred_mean = output[:, 0]
                pred_logvar = output[:, 1]
                loss = gaussian_nll_loss(pred_mean, pred_logvar, label_batch.squeeze())
                val_loss += loss.item()

        val_loss /= len(val_loader)
        scheduler.step(val_loss)

        print(f"✅ Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # ✅ Save model
        torch.save(model.state_dict(), f"EffNetV2S_EarCount_Conf_E{epoch+1}.pth")


In [3]:
train_df, val_df, test_df = loadSplitData("RGB_DSM_totEarNum.csv")
train_loader, val_loader, test_loader = createLoader(train_df, val_df, test_df)

Train Size: 47840, Validation Size: 5980, Test Size: 5980
Train Batches: 2990, Validation Batches: 374, Test Batches: 374


In [4]:
# Initialize
if torch.backends.mps.is_available():
    device = "mps"  # ✅ Use Apple Metal (Mac M1/M2)
    torch.set_default_tensor_type(torch.FloatTensor)
elif torch.cuda.is_available():
    device = "cuda"  # ✅ Use NVIDIA CUDA (Windows RTX 4060)
else:
    device = "cpu"  # ✅ Default to CPU if no GPU is available
print(f"✅ Using device: {device}")

✅ Using device: cuda


In [5]:
EfficientNetV2Model = EfficientNetV2SWheatCountWithConfidence().to(device)
optimizer = optim.Adam(EfficientNetV2Model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
train_model(EfficientNetV2Model, train_loader, val_loader, optimizer, scheduler, device, num_epochs=10)  

Epoch 1/10 | Batch 0/2990 | Loss: 132156.5625
Epoch 1/10 | Batch 20/2990 | Loss: 18.4873
Epoch 1/10 | Batch 40/2990 | Loss: 19.9267
Epoch 1/10 | Batch 60/2990 | Loss: 17.3949
Epoch 1/10 | Batch 80/2990 | Loss: 15.0106
Epoch 1/10 | Batch 100/2990 | Loss: 15.9969
Epoch 1/10 | Batch 120/2990 | Loss: 15.4573
Epoch 1/10 | Batch 140/2990 | Loss: 15.4008
Epoch 1/10 | Batch 160/2990 | Loss: 15.2395
Epoch 1/10 | Batch 180/2990 | Loss: 14.2859
Epoch 1/10 | Batch 200/2990 | Loss: 13.9945
Epoch 1/10 | Batch 220/2990 | Loss: 13.8882
Epoch 1/10 | Batch 240/2990 | Loss: 14.1885
Epoch 1/10 | Batch 260/2990 | Loss: 13.6968
Epoch 1/10 | Batch 280/2990 | Loss: 13.7722
Epoch 1/10 | Batch 300/2990 | Loss: 13.7549
Epoch 1/10 | Batch 320/2990 | Loss: 14.4435
Epoch 1/10 | Batch 340/2990 | Loss: 13.9169
Epoch 1/10 | Batch 360/2990 | Loss: 13.6319
Epoch 1/10 | Batch 380/2990 | Loss: 14.0031
Epoch 1/10 | Batch 400/2990 | Loss: 13.5268
Epoch 1/10 | Batch 420/2990 | Loss: 14.6741
Epoch 1/10 | Batch 440/2990 | Loss

KeyboardInterrupt: 

In [ ]:
# CoAtNetModel = CoAtNetWheatCountWithConfidence().to(device)
# optimizer = optim.Adam(CoAtNetModel.parameters(), lr=1e-4)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
# train_model(CoAtNetModel, train_loader, val_loader, optimizer, scheduler, device, num_epochs=10)